# Ex3 - CreatingDatacube


The notebook shortly demonstrates different ways to create a datacube using `icecube` toolkit. 

In [1]:
import os
import json
import rasterio
import icecube
import glob
from pathlib import Path
from icecube.bin.generate_cube import IceyeProcessGenerateCube

In [2]:
# set paths for the demo
icecube_abspath = str(Path(icecube.__file__).parent.parent)
resource_dir = os.path.join(icecube_abspath, "tests/resources")
grd_dir = os.path.join(resource_dir, "grd_stack/")
cube_save_path = os.path.join(icecube_abspath, "icecube/dataset/temp/my_awesome_cube.nc")
Path(str(Path(cube_save_path).parent)).mkdir(parents=True, exist_ok=True)

vector_labels_fpath = os.path.join(resource_dir, "labels/dummy_vector_labels.json")
masks_labels_fpath = os.path.join(resource_dir, "labels/dummy_mask_labels.json")

**Note:** If you don't have the raster labels saved on your disk, please run the following function and it will generate the labels in the destination folder as specified above. 

In [3]:
from tests.raster_labels_datacube_test import create_run_time_masks_labels
create_run_time_masks_labels()

## icecube CLI

One can use the `icecube` CLI to generate a datacube. Using `icecube --help`, one can see the accepted argument/flags to build the ICEcube. 

Following is the list of positional/optional arguments needed to build the datacube.

- **raster_dir (positional):** path/to/directory where raster are stored 

- **labels-fpath (optional):** path/to/labels.json. The structure of JSON file must be icecube friendly. Please have a look at `icecube.bin.labels_cube.create_json_labels` for more details.

- **cube-save (optional):** path/to/cube.nc where datacube shall be saved

In [4]:
# we can use --help flag to see the accepted args easily
! icecube --help

usage: icecube [-h] [--labels-fpath LABELS_FPATH] [--cube-save CUBE_SAVE]
               raster_dir

CLI support for generating ICEYE datacubes

positional arguments:
  raster_dir            Path/to/directory where raster are stored

optional arguments:
  -h, --help            show this help message and exit
  --labels-fpath LABELS_FPATH
                        path/to/labels.json (in icecube JSON structure) to
                        populate in datacube (Optional)
  --cube-save CUBE_SAVE
                        path/to/cube.nc where datacube shall be saved
                        (Optional)


In [5]:
# Let's build the datacube with only SAR Images
! icecube $grd_dir

09/07/2021 08:16:39 PM - sar_datacube_metadata.py - [INFO] - Building the metadata from the folder /mnt/xor/ICEYE_PACKAGES/icecube/tests/resources/grd_stack/ using GRD
processing rasters for cubes: 100%|██████████████| 3/3 [00:00<00:00, 128.17it/s]
09/07/2021 08:16:39 PM - common_utils.py - [INFO] - create running time is 0.0386 seconds
09/07/2021 08:16:39 PM - generate_cube.py - [INFO] - Skipping labels-cube built, either labels-fpath was not provided or inconsistent extension naming found
09/07/2021 08:16:39 PM - generate_cube.py - [INFO] - Datacube {'Azimuth': 10, 'Band': 3, 'Range': 10} shape built
Generated cube dimensions are: {'Azimuth': 10, 'Band': 3, 'Range': 10}


In [6]:
# building the datacube with SAR data and labels
! icecube $grd_dir --labels-fpath $masks_labels_fpath

09/07/2021 08:16:40 PM - sar_datacube_metadata.py - [INFO] - Building the metadata from the folder /mnt/xor/ICEYE_PACKAGES/icecube/tests/resources/grd_stack/ using GRD
processing rasters for cubes: 100%|██████████████| 3/3 [00:00<00:00, 127.45it/s]
09/07/2021 08:16:40 PM - common_utils.py - [INFO] - create running time is 0.0388 seconds
09/07/2021 08:16:40 PM - sar_datacube_metadata.py - [INFO] - Building the metadata from the folder /mnt/xor/ICEYE_PACKAGES/icecube/tests/resources/grd_stack/ using GRD
/home/iali/anaconda3/envs/icecube_env/lib/python3.8/site-packages/rasterio/__init__.py:207: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
processing rasters for labels cube: 100%|████████| 3/3 [00:00<00:00, 606.81it/s]
09/07/2021 08:16:40 PM - common_utils.py - [INFO] - create running time is 0.0181 seconds
Generated cube dimensions are: {'Azimuth': 10, 'Band': 3, 'R

In [7]:
# building the datacube with SAR data and labels and saving it on the local disk

! icecube $grd_dir --labels-fpath $masks_labels_fpath --cube-save $cube_save_path 
assert(os.path.exists(cube_save_path)) # confirm that file exists.

09/07/2021 08:16:41 PM - sar_datacube_metadata.py - [INFO] - Building the metadata from the folder /mnt/xor/ICEYE_PACKAGES/icecube/tests/resources/grd_stack/ using GRD
processing rasters for cubes: 100%|██████████████| 3/3 [00:00<00:00, 109.96it/s]
09/07/2021 08:16:41 PM - common_utils.py - [INFO] - create running time is 0.0438 seconds
09/07/2021 08:16:41 PM - sar_datacube_metadata.py - [INFO] - Building the metadata from the folder /mnt/xor/ICEYE_PACKAGES/icecube/tests/resources/grd_stack/ using GRD
/home/iali/anaconda3/envs/icecube_env/lib/python3.8/site-packages/rasterio/__init__.py:207: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
processing rasters for labels cube: 100%|████████| 3/3 [00:00<00:00, 895.84it/s]
09/07/2021 08:16:41 PM - common_utils.py - [INFO] - create running time is 0.0153 seconds
Generated cube dimensions are: {'Azimuth': 10, 'Band': 3, 'R

Please note that CLI method reads `icecube/icecube/config.json` file to read the configuration. This file can be modified to change the cube configuration via CLI as needed

## IceyeProcessGenerateCube

Another way to create datacube is to use `icecube.bin.generate_cube.IceyeProcessGenerateCube` class. It provides useful method `create_cube` to generate datacubes. You can find few workflow examples in the script: `icecube.bin.generate_cube` too that will guide you how to create datacubes.

For this demo, we will use the JSON configuration stored in `icecube/tests/resources/json_config/config_use_case5.json`

In [8]:
# Here is a quick look at how the configuration looks like. 

#{
#    "start_date": 20210425,
#    "end_date" : 20210430,
#    "min_incidence_angle" : 20,
#    "max_incidence_angle" : 34,
#    "temporal_resolution" : 1,
#    "temporal_overlap" : 1
#}

# We are asking the cube generator to take into consideration rasters in our stack that span from the 
# dates 25th April to 30th April, and have incidence angles ranging from 20 degrees to 30 degrees. Also the generated 
# cube should 've a and have temporal resolution of 1 day. Temporal overlap denotes that we will accept 
# rasters from the same dates too. 


In [9]:
cube_config_fpath = os.path.join(resource_dir, "json_config/config_use_case5.json")
dc = IceyeProcessGenerateCube.create_cube(
    grd_dir, cube_config_fpath, masks_labels_fpath
)

dc.to_file(cube_save_path)
assert(os.path.exists(cube_save_path)) # confirm that file exists.

09/07/2021 08:16:41 PM - sar_datacube_metadata.py - [INFO] - Building the metadata from the folder /mnt/xor/ICEYE_PACKAGES/icecube/tests/resources/grd_stack/ using GRD
processing rasters for cubes: 100%|██████████| 6/6 [00:00<00:00, 171.21it/s]
09/07/2021 08:16:41 PM - common_utils.py - [INFO] - create running time is 0.0816 seconds
09/07/2021 08:16:41 PM - sar_datacube_metadata.py - [INFO] - Building the metadata from the folder /mnt/xor/ICEYE_PACKAGES/icecube/tests/resources/grd_stack/ using GRD
/home/iali/anaconda3/envs/icecube_env/lib/python3.8/site-packages/rasterio/__init__.py:207: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
processing rasters for labels cube: 100%|██████████| 6/6 [00:00<00:00, 978.80it/s]
09/07/2021 08:16:41 PM - common_utils.py - [INFO] - create running time is 0.0251 seconds


In [10]:
# we can now see how our datacube looks like:
dc.xrdataset

<xarray.Dataset>
Dimensions:    (Azimuth: 10, Band: 6, Range: 10)
Coordinates:
  * Band       (Band) datetime64[ns] 2021-04-25 2021-04-26 ... 2021-04-30
  * Azimuth    (Azimuth) int64 0 1 2 3 4 5 6 7 8 9
  * Range      (Range) int64 0 1 2 3 4 5 6 7 8 9
Data variables:
    Intensity  (Band, Azimuth, Range) uint16 dask.array<chunksize=(1, 10, 10), meta=np.ndarray>
    Labels     (Band, Azimuth, Range) uint8 dask.array<chunksize=(1, 10, 10), meta=np.ndarray>

Similary feel free to use "vector_labels_fpath" to generate cube as well.

One can also use the following two methods to create the SARdatacube and LabelsDatacube separately. Usually you will not need to interact with these methods unless you are interested in low-level details. 


- `icecube.bin.sar_cube.sar_datacube.SARDatacube.create()` to create SARDatacube
- `icecube.bin.labels_cube.labels_datacube.LabelsDatacube.create()` to create LabelsDatacube    

## Cube Configuration

Cube configuration is a convenient way for the user to build datacubes from a monolithic directory of SAR images without worrying about the manual selection of SAR data. You can dump all SAR images into a common directory and slice the information per your need by specifying the different parameters. 

The following is a brief description of the parameters that can be passed to configure datacubes:

- `start_date`: The date from which the SAR images will be considered for the stack (format: int/str %Y%M%D)
- `end_date`: The last date of the image stack (format: int/str %Y%M%D)
- `min_incidence_angle`: The lowest incidence angle in the image stack (format: float/int)
- `max_incidence_angle`:  The highest incidence angle in the image stack (format: float/int)
- `temporal_resolution`: The observation interval (number of days) to be considered between the images in the stack (format: float/int)
- `temporal_overlap`: To decide whether images with the same date should be considered (format: bool/int)

If the default user configuration is used, all images inside the directory are considered for building the datacubes and no pruning on dates or incidence angles is performed.

Datacubes can be easily configured by changing the parameters inside the JSON configuration file. By default, `icecube/config.json` is used as the configuration file. 

Some example JSON files can be found under `tests/resources/json_config/*.json`.

For comprehensively demonstrating how cube configuration works, we will use GRD stack in our `tests/resources/grd_stack/*.tif`. 

Let's first have a look at some of the important metadata fields of these GRD products, as shown in the cell below, to get a better idea of parameter values to play around with.

In [11]:
grd_fpaths = glob.glob(grd_dir+"*")
for grd_fpath in grd_fpaths:
    m = rasterio.open(grd_fpath).tags()
    print("Product file : {} has acquisition date: {} and incidence angle : {}".format(m["PRODUCT_FILE"],
                                                                                      m["ACQUISITION_END_UTC"],
                                                                                      m["INCIDENCE_CENTER"]))

Product file : ICEYE_GRD_54549_20210427T215124_hollow_10x10pixels_fake_2.tif has acquisition date: 2021-05-27T21:51:30.025535 and incidence angle : 28.5
Product file : ICEYE_GRD_54549_20210427T215124_hollow_10x10pixels_fake_1.tif has acquisition date: 2021-04-27T21:51:30.025535 and incidence angle : 29.5
Product file : ICEYE_GRD_54549_20210427T215124_hollow_10x10pixels_fake_0.tif has acquisition date: 2021-04-28T21:51:30.025535 and incidence angle : 30.5


We can see that we have three GRD products that have acquisition dates from 27th April to 27th of May. Incidence angle ranges from 28.5 deg to 30.5 deg.

### Acquisition Dates

Let's create a sample JSON configuration with the following parameters and save it for creating a datacube within required dates. 

In [12]:
# In this cell, we will create a JSON cofiguration file that constraints our datacube to be within
# certain range of dates only. This can be very useful to analyze stacks within specific dates.  

cube_config_fpath = os.path.join(icecube_abspath, "icecube/dataset/temp/sample_config.json")
cube_config_dic = {}
cube_config_dic['start_date'] = "20210425"
cube_config_dic['end_date'] = "20210430"

with open(cube_config_fpath, 'w') as outfile:
    json.dump(cube_config_dic, outfile)

# if you take a look, this is how our configuration file looks like:

#{
#    "start_date": "20210425",
#    "end_date": "20210430",
#}

In [13]:
# Let's build a datacube using the above configuration file

dc = IceyeProcessGenerateCube.create_cube(
    grd_dir, cube_config_fpath, masks_labels_fpath
)

09/07/2021 08:16:42 PM - sar_datacube_metadata.py - [INFO] - Building the metadata from the folder /mnt/xor/ICEYE_PACKAGES/icecube/tests/resources/grd_stack/ using GRD
processing rasters for cubes: 100%|██████████| 2/2 [00:00<00:00, 223.20it/s]
09/07/2021 08:16:42 PM - common_utils.py - [INFO] - create running time is 0.0224 seconds
09/07/2021 08:16:42 PM - sar_datacube_metadata.py - [INFO] - Building the metadata from the folder /mnt/xor/ICEYE_PACKAGES/icecube/tests/resources/grd_stack/ using GRD
/home/iali/anaconda3/envs/icecube_env/lib/python3.8/site-packages/rasterio/__init__.py:207: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
processing rasters for labels cube: 100%|██████████| 2/2 [00:00<00:00, 922.43it/s]
09/07/2021 08:16:42 PM - common_utils.py - [INFO] - create running time is 0.0149 seconds


In [14]:
dc.xrdataset["Band"]

<xarray.DataArray 'Band' (Band: 2)>
array(['2021-04-27T00:00:00.000000000', '2021-04-28T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * Band     (Band) datetime64[ns] 2021-04-27 2021-04-28

We can see that out of three rasters in the directory, only two rasters have been selected for the datacube as expected as 27th May is out of our specified range of dates.

### Temporal Resolution

Sometimes it is important to see how seasonality factors plays in our stack of images. Be it an event of interest of a short time period or repeated nature, you can configure the datacubes to suit your application needs. This can be very useful in a lot of applications like agricultural analysis and harvest detection.

By setting `temporal_resolution` parameter in `config.json`, one can analysis the stack of images after `d` days. 

In [15]:
# let's use the same configuration except we will set temporal_resolution to be 1 in our configuration.

cube_config_fpath = os.path.join(icecube_abspath, "icecube/dataset/temp/sample_config.json")
cube_config_dic = {}
cube_config_dic['start_date'] = "20210425"
cube_config_dic['end_date'] = "20210430"
cube_config_dic['temporal_resolution'] = 1

with open(cube_config_fpath, 'w') as outfile:
    json.dump(cube_config_dic, outfile)

# if you take a look, this is how our configuration file looks like:

#{
#    "start_date": "20210425",
#    "end_date": "20210430",
#    "temporal_resolution": 1
#}

In [16]:
dc = IceyeProcessGenerateCube.create_cube(
    grd_dir, cube_config_fpath, masks_labels_fpath
)

09/07/2021 08:16:42 PM - sar_datacube_metadata.py - [INFO] - Building the metadata from the folder /mnt/xor/ICEYE_PACKAGES/icecube/tests/resources/grd_stack/ using GRD
processing rasters for cubes: 100%|██████████| 6/6 [00:00<00:00, 456.13it/s]
09/07/2021 08:16:42 PM - common_utils.py - [INFO] - create running time is 0.0313 seconds
09/07/2021 08:16:42 PM - sar_datacube_metadata.py - [INFO] - Building the metadata from the folder /mnt/xor/ICEYE_PACKAGES/icecube/tests/resources/grd_stack/ using GRD
/home/iali/anaconda3/envs/icecube_env/lib/python3.8/site-packages/rasterio/__init__.py:207: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
processing rasters for labels cube: 100%|██████████| 6/6 [00:00<00:00, 1108.92it/s]
09/07/2021 08:16:42 PM - common_utils.py - [INFO] - create running time is 0.0234 seconds


In [17]:
dc.xrdataset

<xarray.Dataset>
Dimensions:    (Azimuth: 10, Band: 6, Range: 10)
Coordinates:
  * Band       (Band) datetime64[ns] 2021-04-25 2021-04-26 ... 2021-04-30
  * Azimuth    (Azimuth) int64 0 1 2 3 4 5 6 7 8 9
  * Range      (Range) int64 0 1 2 3 4 5 6 7 8 9
Data variables:
    Intensity  (Band, Azimuth, Range) uint16 dask.array<chunksize=(1, 10, 10), meta=np.ndarray>
    Labels     (Band, Azimuth, Range) uint8 dask.array<chunksize=(1, 10, 10), meta=np.ndarray>

In [18]:
dc.xrdataset["Intensity"].attrs["product_file"]

['None',
 'None',
 'ICEYE_GRD_54549_20210427T215124_hollow_10x10pixels_fake_1.tif',
 'ICEYE_GRD_54549_20210427T215124_hollow_10x10pixels_fake_0.tif',
 'None',
 'None']

We can see that the number of bands have increased now as expected. Our datacube spans from 25th April to 30th April with a time resolution of 1 day. `None` values denote temporal gaps; clearly our stack is not daily repeat in this case for the specified date range.

### Temporal Overlap

If we have multiple acquisitions from the same date, by default only one acquisition is retained. By setting `temporal_overlap` to be `True`, we can allow multiple acquisitions form the same date to be part of our datacube.

In [19]:
# let's use the same configuration except we will set temporal_resolution to be 1 in our configuration.

cube_config_fpath = os.path.join(icecube_abspath, "icecube/dataset/temp/sample_config.json")
cube_config_dic = {}
cube_config_dic['start_date'] = "20210425"
cube_config_dic['end_date'] = "20210530"
cube_config_dic['temporal_overlap'] = 1 # 1 is True, 0 is False

with open(cube_config_fpath, 'w') as outfile:
    json.dump(cube_config_dic, outfile)

# if you take a look, this is how our configuration file looks like:
#{"start_date": "20210425", "end_date": "20210530", "temporal_overlap": 1}

In [20]:
dc = IceyeProcessGenerateCube.create_cube(
    grd_dir, cube_config_fpath, masks_labels_fpath
)

09/07/2021 08:16:42 PM - sar_datacube_metadata.py - [INFO] - Building the metadata from the folder /mnt/xor/ICEYE_PACKAGES/icecube/tests/resources/grd_stack/ using GRD
processing rasters for cubes: 100%|██████████| 3/3 [00:00<00:00, 223.42it/s]
09/07/2021 08:16:42 PM - common_utils.py - [INFO] - create running time is 0.0288 seconds
09/07/2021 08:16:42 PM - sar_datacube_metadata.py - [INFO] - Building the metadata from the folder /mnt/xor/ICEYE_PACKAGES/icecube/tests/resources/grd_stack/ using GRD
/home/iali/anaconda3/envs/icecube_env/lib/python3.8/site-packages/rasterio/__init__.py:207: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
processing rasters for labels cube: 100%|██████████| 3/3 [00:00<00:00, 986.12it/s]
09/07/2021 08:16:42 PM - common_utils.py - [INFO] - create running time is 0.0167 seconds


In [21]:
dc.xrdataset

<xarray.Dataset>
Dimensions:    (Azimuth: 10, Band: 3, Range: 10)
Coordinates:
  * Band       (Band) datetime64[ns] 2021-04-27 2021-04-28 2021-05-27
  * Azimuth    (Azimuth) int64 0 1 2 3 4 5 6 7 8 9
  * Range      (Range) int64 0 1 2 3 4 5 6 7 8 9
Data variables:
    Intensity  (Band, Azimuth, Range) uint16 329 389 217 418 ... 597 85 482 334
    Labels     (Band, Azimuth, Range) uint8 dask.array<chunksize=(1, 10, 10), meta=np.ndarray>

In [22]:
dc.xrdataset["Intensity"].attrs["product_file"]

['ICEYE_GRD_54549_20210427T215124_hollow_10x10pixels_fake_1.tif',
 'ICEYE_GRD_54549_20210427T215124_hollow_10x10pixels_fake_0.tif',
 'ICEYE_GRD_54549_20210427T215124_hollow_10x10pixels_fake_2.tif']

We can see that we have all of our products now as expected.

### Incidence Angle 

Incidence angle of a SAR image is one its most important feature. Studying how incidence angles affect an application is an interesting and active area of research in the SAR domain. Optimizing incidence angles can help a lot in a range of applications with SAR data. For ML applications with SAR data, incidence angle becomes an important factor to consider too. 

With cube configuration, one can easily select rasters from the stack that have specific range of incidence angles. This can help to build training datasets to perform A/B testing and performance benchmarks, and deduce useful insights.

In [23]:
# We know that our stack spans from 28.5 degrees to 30.5 degrees

cube_config_fpath = os.path.join(icecube_abspath, "icecube/dataset/temp/sample_config.json")
cube_config_dic = {}
cube_config_dic['min_incidence_angle'] = 25 
cube_config_dic['max_incidence_angle'] = 27

with open(cube_config_fpath, 'w') as outfile:
    json.dump(cube_config_dic, outfile)

# if you take a look, this is how our configuration file looks like:
#{"min_incidence_angle": 25, "max_incidence_angle": 27}

In [24]:
try:
    dc = IceyeProcessGenerateCube.create_cube(
        grd_dir, cube_config_fpath, masks_labels_fpath
    )
except:
    print("No rasters found against given configuration. Please check user-configuration.")

09/07/2021 08:16:42 PM - sar_datacube_metadata.py - [INFO] - Building the metadata from the folder /mnt/xor/ICEYE_PACKAGES/icecube/tests/resources/grd_stack/ using GRD


No rasters found against given configuration. Please check user-configuration.


we can see that our datacube built was not successful because we don't have any images in this incidence angle range. Let's try again with a different configuration of incidence angles.

In [25]:
# We know that our stack spans from 28.5 degrees to 30.5 degrees

cube_config_fpath = os.path.join(icecube_abspath, "icecube/dataset/temp/sample_config.json")
cube_config_dic = {}
cube_config_dic['min_incidence_angle'] = 25 
cube_config_dic['max_incidence_angle'] = 29.5

with open(cube_config_fpath, 'w') as outfile:
    json.dump(cube_config_dic, outfile)

# if you take a look, this is how our configuration file looks like:
#{"min_incidence_angle": 25, "max_incidence_angle": 29.5}

In [26]:
dc = IceyeProcessGenerateCube.create_cube(
        grd_dir, cube_config_fpath, masks_labels_fpath
    )

09/07/2021 08:16:42 PM - sar_datacube_metadata.py - [INFO] - Building the metadata from the folder /mnt/xor/ICEYE_PACKAGES/icecube/tests/resources/grd_stack/ using GRD
processing rasters for cubes: 100%|██████████| 2/2 [00:00<00:00, 216.82it/s]
09/07/2021 08:16:42 PM - common_utils.py - [INFO] - create running time is 0.0222 seconds
09/07/2021 08:16:42 PM - sar_datacube_metadata.py - [INFO] - Building the metadata from the folder /mnt/xor/ICEYE_PACKAGES/icecube/tests/resources/grd_stack/ using GRD
/home/iali/anaconda3/envs/icecube_env/lib/python3.8/site-packages/rasterio/__init__.py:207: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
processing rasters for labels cube: 100%|██████████| 2/2 [00:00<00:00, 949.47it/s]
09/07/2021 08:16:42 PM - common_utils.py - [INFO] - create running time is 0.0142 seconds


In [27]:
dc.xrdataset

<xarray.Dataset>
Dimensions:    (Azimuth: 10, Band: 2, Range: 10)
Coordinates:
  * Band       (Band) datetime64[ns] 2021-04-27 2021-05-27
  * Azimuth    (Azimuth) int64 0 1 2 3 4 5 6 7 8 9
  * Range      (Range) int64 0 1 2 3 4 5 6 7 8 9
Data variables:
    Intensity  (Band, Azimuth, Range) uint16 329 389 217 418 ... 597 85 482 334
    Labels     (Band, Azimuth, Range) uint8 dask.array<chunksize=(1, 10, 10), meta=np.ndarray>

In [28]:
dc.xrdataset["Intensity"].attrs["product_file"]

['ICEYE_GRD_54549_20210427T215124_hollow_10x10pixels_fake_1.tif',
 'ICEYE_GRD_54549_20210427T215124_hollow_10x10pixels_fake_2.tif']

We can confirm using the metadata above that built datacube is indeed within our specified angle range. 

In [29]:
# Similary one can use combination of these parameters to configure datacubes as needed. 
# A sample configuration could look like this:

# {
# "start_date": "20210425", 
# "end_date": "20210530", 
# "temporal_resolution": 1, 
# "min_incidence_angle": 25, 
# "max_incidence_angle": 29.5
# }

We leave it upto the user to try this specific configuration and hopefully some other configurations too.

**Happy Cubing!**